<h2>Cleaning and Merging Several Datasets to Analyze Per Capita Homelessness by Geography</h2>

<h3>Background</h3>

Warning: this code is not an analysis. Instead, it is the source code to create the dataset behind [this dashboard](https://public.tableau.com/profile/colin.sorensen#!/vizhome/HomelessnessPerCapita_15899979797080/HomelessnessPerCapita) showing homelessness per capita across the United States.

I'm including this as a demonstration of skills with cleaning and combining several disparate files to create a usable dataset for analysis. After all--they say it's 80% of any data professional's job!

<h3>Cleaning/Merging Data</h3>

In [1]:
cd /Users/colinsorensen/Desktop/data/Homelessness_pc

/Users/colinsorensen/Desktop/data/Homelessness_pc


In [2]:
import pandas as pd
import numpy as np

Instead of importing files from my harddrive or the cloud, they will be loaded directly from HUD and the Census Bureau's websites. The first several datasets from HUD include 10 years of [Point-in-Time Count](https://endhomelessness.org/resource/what-is-a-point-in-time-count/) data from across the country, which includes estimates of the total number of individuals experiencing homelessness in each community as measured through an annual count.

In [3]:
pit = 'https://www.hudexchange.info/resources/documents/2007-2019-PIT-Counts-by-CoC.xlsx'

#Read Point-in-Time files
pit10=pd.read_excel(pit, '2010')
pit11=pd.read_excel(pit, '2011')
pit12=pd.read_excel(pit, '2012')
pit13=pd.read_excel(pit, '2013')
pit14=pd.read_excel(pit, '2014')
pit15=pd.read_excel(pit, '2015')
pit16=pd.read_excel(pit, '2016')
pit17=pd.read_excel(pit, '2017')
pit18=pd.read_excel(pit, '2018')
pit19=pd.read_excel(pit, '2019')

The next dataset is from the Census Bureau and shows total county populations from all counties across the country.

In [4]:
#Import County Census file
census = 'https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/co-est2019-alldata.csv'
county_pops = pd.read_csv(census,encoding = 'ISO-8859-1')

Homeless data is not kept at the county-level, but insead at the 'Continuum of Care' (or CoC) level. These are HUD-defined geographic regions that consist of singular or multi-county boundaries. Because the HUD dataset only reports numbers for each CoC and not for each county, the below dataset provides information about which counties belong to which CoCs. For example: Barbour County, Bibb County, and Blount County all belong to the CoC AL-507 (see below).

In [5]:
#Import crosswalk file
crosswalk = pd.read_csv('crosswalk.csv')

crosswalk.head()

,ST,State,County State,County,CoC Number,FIPS ID
0,AL,Alabama,"Autauga County, Alabama",Autauga,AL-504,1001.0
1,AL,Alabama,"Baldwin County, Alabama",Baldwin,AL-501,1003.0
2,AL,Alabama,"Barbour County, Alabama",Barbour,AL-507,1005.0
3,AL,Alabama,"Bibb County, Alabama",Bibb,AL-507,1007.0
4,AL,Alabama,"Blount County, Alabama",Blount,AL-507,1009.0


Next, every single column of each PIT dataframe includes a year (e.g. 'Unsheltered Homelessness, 2015'). This is unnecessary and makes the columns too long, so I'll strip the comma from each column name and everything that comes after (i.e. the year).

In [6]:
pit_dfs=[pit10,pit11,pit12,pit13,pit14,pit15,pit16,pit17,pit18,pit19]

for df in pit_dfs:
    df.columns = df.columns.str.split(',').str[0]

Instead, it makes more sense to create a new 'year' column with the year included on each row.

In [7]:
#Add year column to each df.
base_year = 2010

for idx, df in enumerate(pit_dfs):
    df['year'] = base_year + idx

With that done, it's time to combine the dataframes into a new dataframe, so that records from 2010 are on top and records from 2019 are on bottom.

In [8]:
since_2010 = pd.concat(pit_dfs)

The original files consisted of way too many unnecessary columns, so the next cell drops keeps only the most important columns.

In [9]:
since_2010 = since_2010[['year', 'CoC Number', 'CoC Name', 'Overall Homeless',
                         'Sheltered Total Homeless','Unsheltered Homeless']]

since_2010.head()

,year,CoC Number,CoC Name,Overall Homeless,Sheltered Total Homeless,Unsheltered Homeless
0,2010,AK-500,Anchorage CoC,1231.0,1113.0,118.0
1,2010,AK-501,Alaska Balance of State CoC,632.0,558.0,74.0
2,2010,AL-500,"Birmingham/Jefferson, St. Clair, Shelby Counti...",2273.0,1069.0,1204.0
3,2010,AL-501,Mobile City & County/Baldwin County CoC,883.0,482.0,401.0
4,2010,AL-502,Florence/Northwest Alabama CoC,170.0,162.0,8.0


In [10]:
since_2010.columns

Index(['year', 'CoC Number', 'CoC Name', 'Overall Homeless',
       'Sheltered Total Homeless', 'Unsheltered Homeless'],
      dtype='object')

Next, it's important to look for any entries in CoC Number that do not match an expected pattern, since this will be the column used to merge dataframes in an upcoming step.

In [11]:
#Convert all rows in 'CoC Number' column to strings
since_2010['CoC Number'] = since_2010['CoC Number'].astype(str)

#Create match pattern: 2 letters, followed by a dash, followed by 3 numbers (e.g. AL-507)
matchpattern = '^[A-Z]{2}-\d{3}$'

#Search for any records that do not match this pattern
since_2010[~since_2010['CoC Number'].str.contains(matchpattern)].head()

,year,CoC Number,CoC Name,Overall Homeless,Sheltered Total Homeless,Unsheltered Homeless
213,2010,MO-604a,"Kansas City, Independence, Lee’s Summit/Jackso...",2153.0,1946.0,207.0
391,2010,nan,Total,637077.0,403543.0,233534.0
392,2010,nan,NaN,NaN,NaN,NaN
393,2010,a MO-604 covers territory in both Missouri and...,NaN,NaN,NaN,NaN
213,2011,MO-604a,"Kansas City, Independence, Lee’s Summit/Jackso...",2992.0,2020.0,972.0


This reveals a few unusual rows. After looking at the original files, HUD included some totals and footnotes at the bottom of each file. Since they do not have CoC numbers, these rows can be dropped.

More importantly, this investigation revealed an MO-604a--I assume because Kansas City CoC is split into two states (Kansas and Missouri). In this case, we'll need to go track down MO-604 to see how to handle this.

In [12]:
#Search for MO-604
since_2010[since_2010['CoC Number'].str.contains('MO-604')].head(10)

,year,CoC Number,CoC Name,Overall Homeless,Sheltered Total Homeless,Unsheltered Homeless
213,2010,MO-604a,"Kansas City, Independence, Lee’s Summit/Jackso...",2153.0,1946.0,207.0
393,2010,a MO-604 covers territory in both Missouri and...,NaN,NaN,NaN,NaN
213,2011,MO-604a,"Kansas City, Independence, Lee’s Summit/Jackso...",2992.0,2020.0,972.0
394,2011,a MO-604 covers territory in both Missouri and...,NaN,NaN,NaN,NaN
212,2012,MO-604a,"Kansas City, Independence, Lee’s Summit/Jackso...",2705.0,1876.0,829.0
393,2012,a MO-604 covers territory in both Missouri and...,NaN,NaN,NaN,NaN
213,2013,MO-604a,"Kansas City, Independence, Lee’s Summit/Jackso...",2691.0,1766.0,925.0
394,2013,a MO-604 covers territory in both Missouri and...,NaN,NaN,NaN,NaN
213,2014,MO-604a,"Kansas City, Independence, Lee’s Summit/Jackso...",2193.0,1918.0,275.0
393,2014,a MO-604 covers territory in both Missouri and...,NaN,NaN,NaN,NaN


In a strange twist, there's a MO-604a but not an MO-604. Odd. in that case, we'll just convert MO-604a to match the rest of the CoCs.

In [13]:
#Rename all MO-604a records as MO-604
since_2010 = since_2010.replace({'MO-604a':'MO-604'}, regex=True)

In addition, it's important to keep only the rows with data and in turn dropping the blank/footnote rows at the bottom of the dataset. This can be done using the same match pattern.

In [14]:
#Keep only those rows that comply with the match pattern
mask = since_2010['CoC Number'].str.contains(matchpattern)
since_2010 = since_2010[mask]

With the PIT dataframe ready to go, the county population data is up next. The plan is to merge it with the crosswalk dataset on a 'County State' column, but this column has to be created first.

In [15]:
#Keep only the most relevant columns
county_pops = county_pops.iloc[:,5:19]

#Combine county name and state name for column 'County State'
county_pops['County State'] = county_pops['CTYNAME'] + ', ' + county_pops['STNAME']

Whith that column created, the county population data and the crosswalk file can be merged.

In [16]:
county_info = pd.merge(crosswalk, county_pops, on='County State')

county_info.head()

,ST,State,County State,County,CoC Number,FIPS ID,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019
0,AL,Alabama,"Autauga County, Alabama",Autauga,AL-504,1001.0,Alabama,Autauga County,54571,54597,54773,55227,54954,54727,54893,54864,55243,55390,55533,55869
1,AL,Alabama,"Baldwin County, Alabama",Baldwin,AL-501,1003.0,Alabama,Baldwin County,182265,182265,183112,186558,190145,194885,199183,202939,207601,212521,217855,223234
2,AL,Alabama,"Barbour County, Alabama",Barbour,AL-507,1005.0,Alabama,Barbour County,27457,27455,27327,27341,27169,26937,26755,26283,25806,25157,24872,24686
3,AL,Alabama,"Bibb County, Alabama",Bibb,AL-507,1007.0,Alabama,Bibb County,22915,22915,22870,22745,22667,22521,22553,22566,22586,22550,22367,22394
4,AL,Alabama,"Blount County, Alabama",Blount,AL-507,1009.0,Alabama,Blount County,57322,57322,57376,57560,57580,57619,57526,57526,57494,57787,57771,57826


Currently each year is represented in a different column, but in order to merge this file with the PIT data, it has to be tidy. Thus, the next step is melting the dataframe so that all years are in one column 'year'.

While we're at it, we'll also strip 'POPESTIMATE' from each cell so that it just shows the year.

In [17]:
#Create list of columns
pop_cols = ['POPESTIMATE2010', 'POPESTIMATE2011','POPESTIMATE2012', 'POPESTIMATE2013', 'POPESTIMATE2014','POPESTIMATE2015', 
            'POPESTIMATE2016', 'POPESTIMATE2017','POPESTIMATE2018', 'POPESTIMATE2019']

#Melt the datame using those columns
pop_melt = pd.melt(frame=county_info, id_vars=['County State','CoC Number'], value_vars=pop_cols,var_name='Year',value_name='Pop Estimate')

In [18]:
#Remove 'POPESTIMATE' from every row and just keep the year
pop_melt['Year'] = pop_melt['Year'].str.strip('POPESTIMATE')

pop_melt.head()

,County State,CoC Number,Year,Pop Estimate
0,"Autauga County, Alabama",AL-504,2010,54773
1,"Baldwin County, Alabama",AL-501,2010,183112
2,"Barbour County, Alabama",AL-507,2010,27327
3,"Bibb County, Alabama",AL-507,2010,22870
4,"Blount County, Alabama",AL-507,2010,57376


Next, we'll create a new column/ID for pop_melt (all of the population information for all counties) and since_2010 (all of the homeless point in time county information for all of the CoCs) and use it to merge both dateframes together.

In [19]:
#Create 'cocnum_year' column for both pop_melt and since_2010
pop_melt['cocnum_year'] = pop_melt['CoC Number'] + pop_melt['Year']
since_2010['cocnum_year'] = since_2010['CoC Number'] + since_2010['year'].astype(str)

#Merge the dataframes
final_df = pd.merge(pop_melt, since_2010, on='cocnum_year')

final_df.head()

,County State,CoC Number_x,Year,Pop Estimate,cocnum_year,year,CoC Number_y,CoC Name,Overall Homeless,Sheltered Total Homeless,Unsheltered Homeless
0,"Autauga County, Alabama",AL-504,2010,54773,AL-5042010,2010,AL-504,Montgomery City & County CoC,444.0,294.0,150.0
1,"Bullock County, Alabama",AL-504,2010,10876,AL-5042010,2010,AL-504,Montgomery City & County CoC,444.0,294.0,150.0
2,"Elmore County, Alabama",AL-504,2010,79552,AL-5042010,2010,AL-504,Montgomery City & County CoC,444.0,294.0,150.0
3,"Lowndes County, Alabama",AL-504,2010,11289,AL-5042010,2010,AL-504,Montgomery City & County CoC,444.0,294.0,150.0
4,"Montgomery County, Alabama",AL-504,2010,229505,AL-5042010,2010,AL-504,Montgomery City & County CoC,444.0,294.0,150.0


It looks like we now have the two columns we need to calculate per capita homelessness--'Overall Homeless' and 'Pop Estimate.' However, this is deceiving because Overall Homeless is at the CoC level and Pop Estimate is at the county level. In cases where the CoC only consists of one county (e.g. San Francisco is CA-501), this calculation would work. However, many CoCs consist of two or more counties, and some have upwards of 40.

Therefore, before we can make any calculations, we must first aggregate each county population as a CoC-wide population total. This can be done through a pivot table.

In [20]:
#Aggregate county populations, grouping by CoC
coc_pop = final_df.pivot_table(index='cocnum_year',values='Pop Estimate',aggfunc=sum)

Finally, we can merge these new CoC totals with final_df. Now there are two columns: 'Pop Estimate_x' represents the county's population and Pop Estimate_y represents the CoC's total population. 

Using the latter, we can create a new column that calculates per capita homelessness by dividing 'Overall Homeless' by 'Pop_Estimate_y'.

In [21]:
#Merge final_Df and coc_pop
final_df = pd.merge(final_df, coc_pop, on='cocnum_year')

In [22]:
#Divide 'Overall Homeless' by 'Pop Estimate_y' to calculate per capita homelessness at the CoC level
final_df['homeless_pc'] = final_df['Overall Homeless'] / final_df['Pop Estimate_y']

While it's almost all finished, the last step is to clean up the final dataframe/column names a bit. This includes a) deleting any duplicative columns that may have resulted from mergin, b) renaming columns, c) making each column name lowercase, and d) replacing spaces in column names with underscores.

In [23]:
#Drop columns
final_df = final_df.drop(['CoC Number_y','year'],axis=1)

#Rename columns
final_df = final_df.rename(columns={'CoC Number_x':'CoC number',
                                    'Pop Estimate_x':'County Population',
                                    'Pop Estimate_y':'CoC Population',
                                    'Overall Homeless':'Total Homeless',
                                    'Sheltered Total Homeless':'Sheltered Homeless'})

#Make all columns lowercase
final_df.columns = map(str.lower, final_df.columns)

#Replace spaces with underscores
final_df.columns = final_df.columns.str.replace(' ', '_')

See below for the final result. It's time to do some analysis!

In [24]:
final_df.head()

,county_state,coc_number,year,county_population,cocnum_year,coc_name,total_homeless,sheltered_homeless,unsheltered_homeless,coc_population,homeless_pc
0,"Autauga County, Alabama",AL-504,2010,54773,AL-5042010,Montgomery City & County CoC,444.0,294.0,150.0,385995,0.00115
1,"Bullock County, Alabama",AL-504,2010,10876,AL-5042010,Montgomery City & County CoC,444.0,294.0,150.0,385995,0.00115
2,"Elmore County, Alabama",AL-504,2010,79552,AL-5042010,Montgomery City & County CoC,444.0,294.0,150.0,385995,0.00115
3,"Lowndes County, Alabama",AL-504,2010,11289,AL-5042010,Montgomery City & County CoC,444.0,294.0,150.0,385995,0.00115
4,"Montgomery County, Alabama",AL-504,2010,229505,AL-5042010,Montgomery City & County CoC,444.0,294.0,150.0,385995,0.00115
